In [21]:
import numpy as np
import pandas as pd
import scipy
import sklearn as skl
# from tqdm.notebook import tqdm

from tqdm import tqdm
import rich


In [22]:
import kagglehub
import pathlib

path = kagglehub.dataset_download("shubhammehta21/movie-lens-small-latest-dataset")

movies_df = pd.read_csv(pathlib.Path(path) / "movies.csv")
ratings_df = pd.read_csv(pathlib.Path(path) / "ratings.csv")


In [23]:
movies_df


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [24]:
ratings_df


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [25]:
ratings_df["rating"].describe()


count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [26]:
sparce_ratings_df = (
    ratings_df.pivot_table(index="userId", columns="movieId", values="rating")
    .fillna(0)
    .astype(pd.SparseDtype("float", fill_value=0.0))
)
sparce_ratings_df


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
from sklearn.model_selection import train_test_split

R_train, R_test = train_test_split(
    ratings_df,
    test_size=0.2,
)


In [28]:
R_train = (
    R_train.pivot_table(index="userId", columns="movieId", values="rating")
    .fillna(0)
    .astype(pd.SparseDtype("float", fill_value=0.0))
)

R_test = (
    R_test.pivot_table(index="userId", columns="movieId", values="rating")
    .fillna(0)
    .astype(pd.SparseDtype("float", fill_value=0.0))
)


In [31]:
import scipy.sparse


class CollaborateFiltrator:  # latent factor model
    def fit(
        self,
        R: scipy.sparse.csr_matrix,
        R_test: scipy.sparse.csr_matrix,
        k: int = 13,
        epochs: int = 20,
    ) -> np.ndarray:
        m, n = R.shape

        rows, cols = R.nonzero()

        eta, lambda_ = 0.01, 0.1

        P = np.random.rand(m, k) * 0.01
        Q = np.random.rand(n, k) * 0.01
        e: scipy.sparse.csr_matrix = R.copy()

        for epoch in range(epochs):
            with tqdm(
                total=len(rows),
                desc=f"Epoch {epoch + 1}/{epochs}",
                ncols=150,
                colour="cyan",
                leave=True,
            ) as epoch_bar:
                for user_id, object_id in zip(rows, cols):
                    e[user_id, object_id] = R[user_id, object_id] - np.dot(
                        P[user_id, :], Q[object_id, :]
                    )

                    p, q = P[user_id, :].copy(), Q[object_id, :].copy()

                    P[user_id, :] += eta * (e[user_id, object_id] * q - lambda_ * p)
                    Q[object_id, :] += eta * (e[user_id, object_id] * p - lambda_ * q)
                    epoch_bar.update(1)
                else:
                    loss = (e.multiply(e)).sum() + (
                        np.linalg.norm(P) ** 2 + np.linalg.norm(Q) ** 2
                    )
                    self.P = P
                    self.Q = Q

                    r_pred = self.predict(*R_test.nonzero())

                    mae = (R_test - r_pred).__abs__().sum() / (R.count_nonzero())
                    rmse = np.sqrt(
                        (R_test - r_pred).multiply(R_test - r_pred).sum()
                        / R.count_nonzero()
                    )

                    epoch_bar.set_postfix(
                        {
                            "loss": f"{loss:.6f}",
                            "MAE": f"{mae:.6f}",
                            "RMSE": f"{rmse:.6f}",
                        }
                    )
                    epoch_bar.update(0)

        self.P = P
        self.Q = Q
        self.m = m
        self.user_count = self.m
        self.n = n
        self.object_count = self.n
        self.k = k
        self.latent_space = self.k

    def predict(
        self, user_ids: list[int] | np.ndarray, object_ids: list[int] | np.ndarray
    ) -> np.ndarray:
        score = np.zeros(shape=(len(user_ids),), dtype=np.float32)
        for idx, [user_id, object_id] in enumerate(zip(user_ids, object_ids)):
            score[idx] = np.dot(self.P[user_id], self.Q[object_id])
        return scipy.sparse.csr_matrix(
            (
                score,
                (
                    np.array(user_ids, dtype=np.int32),
                    np.array(object_ids, dtype=np.int32),
                ),
            ),
            dtype=np.float32,
        )


spread = 5 - 1


filtrator = CollaborateFiltrator()
filtrator.fit(
    R_train.sparse.to_coo().tocsr(),
    R_test.sparse.to_coo().tocsr(),
    k=37,
    epochs=23,
)


Epoch 23/23: 100%|███████████████████████████████████████████| 80668/80668 [00:04<00:00, 18793.65it/s, loss=76891.150738, MAE=0.285226, RMSE=0.709269]


In [33]:
R_test


movieId,1,2,3,4,5,6,7,9,10,11,...,185135,187541,187593,187595,187717,188675,189333,193573,193579,193581
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[["userId", "movieId", "rating"]], reader)

# Обучение и оценка
surprise_filtrator = SVD(
    n_factors=37,
    biased=True,
)
print(
    cross_validate(
        surprise_filtrator,
        data,
        measures=["RMSE", "MAE"],
        cv=5,
        verbose=True,
    )
)


TypeError: cross_validate() got an unexpected keyword argument 'epoch'

In [47]:
print(SVD.__doc__)


The famous *SVD* algorithm, as popularized by `Simon Funk
    <https://sifter.org/~simon/journal/20061211.html>`_ during the Netflix
    Prize. When baselines are not used, this is equivalent to Probabilistic
    Matrix Factorization :cite:`salakhutdinov2008a` (see :ref:`note
    <unbiased_note>` below).

    The prediction :math:`\hat{r}_{ui}` is set as:

    .. math::
        \hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u

    If user :math:`u` is unknown, then the bias :math:`b_u` and the factors
    :math:`p_u` are assumed to be zero. The same applies for item :math:`i`
    with :math:`b_i` and :math:`q_i`.

    For details, see equation (5) from :cite:`Koren:2009`. See also
    :cite:`Ricci:2010`, section 5.3.1.

    To estimate all the unknown, we minimize the following regularized squared
    error:

    .. math::
        \sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
        \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)


    The minimization is perf